In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
import time
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)
#https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html

In [2]:
propiedades = pd.read_csv('../../set_datos_propiedades.csv')

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

# LightGBM

In [9]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [10]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [29]:
lgbm = lgb.LGBMRegressor()
lgbm.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = lgbm.predict(set_pruebas.loc[:,columnas])
precision = lgbm.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {}%, error = {}".format(precision, error))

Precision = 0.230676213694%, error = 1.37641955262e+11


### Ahora que tenemos una intuicion, probamos variando los parametros

In [30]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [33]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

res = []

In [ ]:
boosting = ['gbdt', 'dart', 'goss', 'rf']
estimators = [10, 20, 50]
learning_rate = [0.1, 0.3, 0.5]

for b in boosting:
    for e in estimators:
        for lr in learning_rate:
            try:
                lgbm = lgb.LGBMRegressor(boosting_type = b, n_estimators = e, learning_rate = lr)
                lgbm.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
                set_pruebas.loc[:,'resultado'] = lgbm.predict(set_pruebas.loc[:,columnas])
                precision = lgbm.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
                error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
                res.append((b, e, lr, precision, error))
                print(b, '-', e, '-', lr)
            except:
                continue

In [36]:
for r in res:
    print ("boosting_type = {}, n_estimators = {}, learnig_rate = {}, precision = {:.2f} % , error = {}".format(r[0],r[1],r[2],r[3], r[4]))

boosting_type = gbdt, n_estimators = 10, learnig_rate = 0.1, precision = 0.23 % , error = 1.37641955262e+11
boosting_type = gbdt, n_estimators = 10, learnig_rate = 0.3, precision = 0.23 % , error = 1.37641955262e+11
boosting_type = gbdt, n_estimators = 10, learnig_rate = 0.5, precision = 0.23 % , error = 1.37641955262e+11
boosting_type = gbdt, n_estimators = 20, learnig_rate = 0.1, precision = 0.46 % , error = 1.37318841383e+11
boosting_type = gbdt, n_estimators = 20, learnig_rate = 0.3, precision = 0.46 % , error = 1.37318841383e+11
boosting_type = gbdt, n_estimators = 20, learnig_rate = 0.5, precision = 0.46 % , error = 1.37318841383e+11
boosting_type = gbdt, n_estimators = 50, learnig_rate = 0.1, precision = 1.17 % , error = 1.36348628327e+11
boosting_type = gbdt, n_estimators = 50, learnig_rate = 0.3, precision = 1.17 % , error = 1.3634861732e+11
boosting_type = gbdt, n_estimators = 50, learnig_rate = 0.5, precision = 1.17 % , error = 1.36348603281e+11
boosting_type = dart, n_estim

In [37]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[4] < min_error:
        min_error = r[4]
        tupla_min_error = r
    if r[3] > max_precision:
        max_precision = r[3]
        tupla_max_precision = r
        
print("Mayor precision = boosting_type = {}, n_estimators = {}, learnig_rate = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3], tupla_max_precision[4]))
print("Menor error = boosting_type = {}, n_estimators = {}, learnig_rate = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3], tupla_min_error[4]))

Mayor precision = boosting_type = goss, n_estimators = 50, learnig_rate = 0.5, precision = 1.19 % , error = 1.36317175083e+11
Menor error = boosting_type = goss, n_estimators = 50, learnig_rate = 0.5, precision = 1.19 % , error = 1.36317175083e+11


# Modifico max_depth

In [38]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio','aire','pileta','cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [39]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [40]:
max_depth = [1, 5, 10, 15]

In [ ]:
for d in max_depth:
    lgbm = lgb.LGBMRegressor(boosting_type = 'goss', n_estimators = 50, learning_rate = 0.5, max_depth = d)
    lgbm.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'resultado'] = lgbm.predict(set_pruebas.loc[:,columnas])
    precision = lgbm.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
    error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
    res.append((b, e, lr, precision, error))
    print(b, '-', e, '-', lr)

In [42]:
for r in res[27:]:
    print ("boosting_type = {}, n_estimators = {}, learnig_rate = {}, precision = {:.2f} % , error = {}".format(r[0],r[1],r[2],r[3], r[4]))

boosting_type = rf, n_estimators = 50, learnig_rate = 0.5, precision = 1.09 % , error = 1.36452039915e+11
boosting_type = rf, n_estimators = 50, learnig_rate = 0.5, precision = 1.18 % , error = 1.36326866928e+11
boosting_type = rf, n_estimators = 50, learnig_rate = 0.5, precision = 1.19 % , error = 1.36317175083e+11
boosting_type = rf, n_estimators = 50, learnig_rate = 0.5, precision = 1.19 % , error = 1.36317175083e+11


## Aumentamos los valores de max depth y n_estimators para ver si mejoramos la precisión

In [47]:
lgbm = lgb.LGBMRegressor(boosting_type = 'goss', n_estimators = 5000, learning_rate = 2.9, max_depth = 1000)
lgbm.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = lgbm.predict(set_pruebas.loc[:,columnas])
precision = lgbm.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {}%, error = {}".format(precision, error))

Precision = 38.9816250348%, error = 84181070076.1


In [ ]:
n_estimators = [5000, 10000, 50000, 100000]
b = 'goss'
lr = 2.9
d = 1000
res = []

for n in n_estimators:
    lgbm = lgb.LGBMRegressor(boosting_type = b, n_estimators = n, learning_rate = lr, max_depth = d)
    lgbm.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'resultado'] = lgbm.predict(set_pruebas.loc[:,columnas])
    precision = lgbm.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
    error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
    res.append((b, n, lr, precision, error))
    print(n)

In [52]:
for r in res:
    print ("boosting_type = {}, n_estimators = {}, learnig_rate = {}, precision = {:.2f} % , error = {}".format(r[0],r[1],r[2],r[3], r[4]))

boosting_type = goss, n_estimators = 5000, learnig_rate = 2.9, precision = 38.98 % , error = 84181070076.1
boosting_type = goss, n_estimators = 10000, learnig_rate = 2.9, precision = 47.01 % , error = 73107356570.7
boosting_type = goss, n_estimators = 50000, learnig_rate = 2.9, precision = 54.98 % , error = 62112893806.8
boosting_type = goss, n_estimators = 100000, learnig_rate = 2.9, precision = 58.98 % , error = 56587427148.9


#### Se puede ver que al aumentar la cantidad de estimadores mejora la precision por lo que tomamos un número más grande

# Calculamos los verdaderos datos

In [53]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")

In [55]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: lgbm.predict([x])[0],axis = 1)

In [56]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.471407e+05
std      2.493879e+05
min     -5.653215e+05
25%      1.145887e+05
50%      1.621351e+05
75%      2.909809e+05
max      8.571289e+06
Name: price_usd, dtype: float64

In [57]:
analizar.loc[:, 'price_usd'] = analizar.loc[:, 'price_usd'].apply(lambda x: abs(x))

In [58]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.477323e+05
std      2.488002e+05
min      6.880918e+03
25%      1.147296e+05
50%      1.624316e+05
75%      2.914919e+05
max      8.571289e+06
Name: price_usd, dtype: float64

In [59]:
resultado = analizar.loc[:,['id','price_usd']]

In [60]:
resultado.to_csv('resultados/lightgbm_resultados.csv', index = False)

## Ahora uso lat y lon en lugar de place name

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()),\
                             ['lat', 'lon', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [5]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [66]:
lgbm = lgb.LGBMRegressor()
lgbm.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = lgbm.predict(set_pruebas.loc[:,columnas])
precision = lgbm.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {}%, error = {}".format(precision, error))

Precision = 0.225789592842%, error = 1.15522485637e+11


In [67]:
res = []

In [ ]:
boosting = ['gbdt', 'dart', 'goss', 'rf']
estimators = [10, 20, 50]
learning_rate = [0.1, 0.3, 0.5]

for b in boosting:
    for e in estimators:
        for lr in learning_rate:
            try:
                lgbm = lgb.LGBMRegressor(boosting_type = b, n_estimators = e, learning_rate = lr)
                lgbm.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
                set_pruebas.loc[:,'resultado'] = lgbm.predict(set_pruebas.loc[:,columnas])
                precision = lgbm.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
                error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
                res.append((b, e, lr, precision, error))
                print(b, '-', e, '-', lr)
            except:
                continue

In [69]:
for r in res:
    print ("boosting_type = {}, n_estimators = {}, learnig_rate = {}, precision = {:.2f} % , error = {}".format(r[0],r[1],r[2],r[3], r[4]))

boosting_type = gbdt, n_estimators = 10, learnig_rate = 0.1, precision = 0.23 % , error = 1.15522485637e+11
boosting_type = gbdt, n_estimators = 10, learnig_rate = 0.3, precision = 0.23 % , error = 1.15522485637e+11
boosting_type = gbdt, n_estimators = 10, learnig_rate = 0.5, precision = 0.23 % , error = 1.15522485637e+11
boosting_type = gbdt, n_estimators = 20, learnig_rate = 0.1, precision = 0.48 % , error = 1.15225738506e+11
boosting_type = gbdt, n_estimators = 20, learnig_rate = 0.3, precision = 0.48 % , error = 1.15225738506e+11
boosting_type = gbdt, n_estimators = 20, learnig_rate = 0.5, precision = 0.48 % , error = 1.15225738506e+11
boosting_type = gbdt, n_estimators = 50, learnig_rate = 0.1, precision = 1.24 % , error = 1.14345223916e+11
boosting_type = gbdt, n_estimators = 50, learnig_rate = 0.3, precision = 1.24 % , error = 1.14345223916e+11
boosting_type = gbdt, n_estimators = 50, learnig_rate = 0.5, precision = 1.24 % , error = 1.14345223916e+11
boosting_type = dart, n_esti

In [70]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[4] < min_error:
        min_error = r[4]
        tupla_min_error = r
    if r[3] > max_precision:
        max_precision = r[3]
        tupla_max_precision = r
        
print("Mayor precision = boosting_type = {}, n_estimators = {}, learnig_rate = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3], tupla_max_precision[4]))
print("Menor error = boosting_type = {}, n_estimators = {}, learnig_rate = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3], tupla_min_error[4]))

Mayor precision = boosting_type = gbdt, n_estimators = 50, learnig_rate = 0.1, precision = 1.24 % , error = 1.14345223916e+11
Menor error = boosting_type = gbdt, n_estimators = 50, learnig_rate = 0.1, precision = 1.24 % , error = 1.14345223916e+11


In [71]:
n_estimators = [5000, 10000, 50000, 100000]
b = 'goss'
lr = 2.9
res = []

for n in n_estimators:
    lgbm = lgb.LGBMRegressor(boosting_type = b, n_estimators = n, learning_rate = lr)
    lgbm.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'resultado'] = lgbm.predict(set_pruebas.loc[:,columnas])
    precision = lgbm.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
    error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
    res.append((b, n, lr, precision, error))
    print(n)

5000
10000
50000
100000


In [72]:
for r in res:
    print ("boosting_type = {}, n_estimators = {}, learnig_rate = {}, precision = {:.2f} % , error = {}".format(r[0],r[1],r[2],r[3], r[4]))

boosting_type = goss, n_estimators = 5000, learnig_rate = 2.9, precision = 39.90 % , error = 69586432759.9
boosting_type = goss, n_estimators = 10000, learnig_rate = 2.9, precision = 49.19 % , error = 58834615136.1
boosting_type = goss, n_estimators = 50000, learnig_rate = 2.9, precision = 62.36 % , error = 43585312292.7
boosting_type = goss, n_estimators = 100000, learnig_rate = 2.9, precision = 68.84 % , error = 36082874388.6


In [73]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[4] < min_error:
        min_error = r[4]
        tupla_min_error = r
    if r[3] > max_precision:
        max_precision = r[3]
        tupla_max_precision = r
        
print("Mayor precision = boosting_type = {}, n_estimators = {}, learnig_rate = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3], tupla_max_precision[4]))
print("Menor error = boosting_type = {}, n_estimators = {}, learnig_rate = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3], tupla_min_error[4]))

Mayor precision = boosting_type = goss, n_estimators = 100000, learnig_rate = 2.9, precision = 68.84 % , error = 36082874388.6
Menor error = boosting_type = goss, n_estimators = 100000, learnig_rate = 2.9, precision = 68.84 % , error = 36082874388.6


# Calculamos los verdaderos datos

In [74]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")

In [75]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: lgbm.predict([x])[0],axis = 1)

In [76]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.389744e+05
std      2.538136e+05
min     -1.128486e+06
25%      1.092698e+05
50%      1.590292e+05
75%      2.817779e+05
max      5.876935e+06
Name: price_usd, dtype: float64

In [77]:
analizar.loc[:, 'price_usd'] = analizar.loc[:, 'price_usd'].apply(lambda x: abs(x))

In [78]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.396352e+05
std      2.531898e+05
min      3.814273e+03
25%      1.093145e+05
50%      1.591093e+05
75%      2.818319e+05
max      5.876935e+06
Name: price_usd, dtype: float64

In [79]:
resultado = analizar.loc[:,['id','price_usd']]

In [80]:
resultado.to_csv('resultados/lightgbm_latlon_resultados.csv', index = False)

# Usando Grid Search y Cross Validation

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [24]:
# Vario unicamente la cantidad de estimadores
boosting = ['goss'] # rf no se tiene en cuenta pues levanta excepcion
estimators = [100, 1000, 5000]
learning_rate = [0.5, 0.9]

parametros = {"n_estimators" : estimators, "learning_rate" : learning_rate, "boosting_type" : boosting}

iteraciones_cross_validation = 25
lgbm = lgb.LGBMRegressor()

In [25]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = lgbm, param_grid = parametros, n_jobs = -1, cv = iteraciones_cross_validation)

grid.fit(set_entrenamiento_datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 12:35:18 --- 19:20:59 
 Precision: 41.32 
 Parametros = {'boosting_type': 'goss', 'learning_rate': 0.5, 'n_estimators': 5000}


In [27]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: grid.predict([x])[0],axis = 1)

analizar.price_usd.describe()

count     14166.000000
mean     233809.103726
std      161628.517309
min       52303.525426
25%      116187.346107
50%      162430.635225
75%      314994.565201
max      753801.382929
Name: price_usd, dtype: float64

In [28]:
resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Lightgbm_GridSearch.csv', index = False)

joblib.dump(grid, 'algoritmos/Lightgbm.pkl')

['algoritmos/Lightgbm.pkl']

#### Para un número mayor de estimadores se debería obtener una mejor precisión pero como el tiempo es muy largo se descarto

# Hago bagging con el mejor algoritmo

In [13]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

set_pruebas = analizar = pd.read_csv("../properati_dataset_modificado.csv")
set_pruebas.loc[:,'price_usd'] = 0.0

In [14]:
cant = 10

for i in range(cant):
    datos = propiedades.sample(frac = 0.4)
    set_entrenamiento_datos = datos.loc[:,columnas]
    set_entrenamiento_resultado = datos.loc[:,'price_aprox_usd']
    
    lgbm = lgb.LGBMRegressor(boosting_type = 'goss', learning_rate = 0.5, n_estimators = 5000) # Mejor resultado
    
    lgbm.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: lgbm.predict([x])[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']

In [17]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Lightgbm_Cross_Validation.csv', index = False)

# Usando SVD con varias dimensiones

In [2]:
propiedades = pd.read_csv('../../set_datos_propiedades.csv')

sup_min = 10
sup_max = 150

lat_min = -35
lat_max = -34

lon_min = -59
lon_max = -58

precio_min = 5000
precio_max = 1000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [7]:
X = propiedades.loc[:,columnas]
y = propiedades.loc[:,'price_aprox_usd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

for i in range(2,len(columnas)):
    svd = TruncatedSVD(n_components = i)
    datos = svd.fit_transform(X_train)
    test = svd.transform(X_test)
    
    lgbm = lgb.LGBMRegressor(n_estimators = 100000)
    lgbm.fit(datos,y_train)
    precision = lgbm.score(test,y_test) * 100
    error = mean_squared_error(lgbm.predict(test),y_test)
    
    print("Dim = {}, Precision = {:.2f}, error = {}, digitos = {}".\
                  format(i,precision,error,len(str(int(round(error,0))))))

Dim = 2, Precision = 41.45, error = 7496075794.63, digitos = 10
Dim = 3, Precision = 71.17, error = 3691564873.03, digitos = 10
Dim = 4, Precision = 74.29, error = 3291071934.07, digitos = 10
Dim = 5, Precision = 74.41, error = 3276746822.98, digitos = 10
Dim = 6, Precision = 75.30, error = 3162544284.26, digitos = 10
Dim = 7, Precision = 76.27, error = 3038492305.81, digitos = 10
Dim = 8, Precision = 76.54, error = 3003914925.53, digitos = 10


In [3]:
set_pruebas = analizar = pd.read_csv("../properati_dataset_modificado.csv")

svd = TruncatedSVD(n_components = 8)
datos = svd.fit_transform(propiedades.loc[:,columnas])
test = svd.transform(set_pruebas.loc[:,columnas])

lgbm = lgb.LGBMRegressor(n_estimators = 100000)
lgbm.fit(datos,propiedades.loc[:,'price_aprox_usd'])
set_pruebas.loc[:,'price_usd'] = lgbm.predict(test)

resultado = set_pruebas.loc[:,['id','price_usd']]
resultado.to_csv('resultados/Lightgbm_SVD.csv', index = False)

# Agregando el resultado de SVD dimension 2 al Dataframe

In [2]:
propiedades = pd.read_csv('../../set_datos_propiedades.csv')

sup_min = 10
sup_max = 150

lat_min = -35
lat_max = -34

lon_min = -59
lon_max = -58

precio_min = 5000
precio_max = 1000000

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2016)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

prop = propiedades.loc[(propiedades.Year >= 2016), :]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [3]:
set_entrenamiento = prop.loc[(prop.Year >= 2016) &((prop.Year < 2017) | (prop.Month < 6))\
                                    ,columnas_precio]
set_pruebas = prop.loc[(prop.Year == 2017) & (prop.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [4]:
svd = TruncatedSVD()
resultado = svd.fit_transform(set_entrenamiento_datos)
resultado = pd.DataFrame(resultado)
pruebas = svd.transform(set_pruebas.loc[:,columnas])
pruebas = pd.DataFrame(pruebas)

In [5]:
set_entrenamiento_datos.reset_index(drop = True, inplace = True)
set_entrenamiento_datos.loc[:,'SVD 1'] = resultado.loc[:,0]
set_entrenamiento_datos.loc[:,'SVD 2'] = resultado.loc[:,1]
set_entrenamiento_datos['SVD 1'].describe()

count    334935.000000
mean         95.098990
std          24.394078
min          52.920358
25%          75.951302
50%          88.599928
75%         110.070518
max         157.152840
Name: SVD 1, dtype: float64

In [6]:
set_pruebas.reset_index(drop = True, inplace = True)
set_pruebas.loc[:,'SVD 1'] = pruebas.loc[:,0]
set_pruebas.loc[:,'SVD 2'] = pruebas.loc[:,1]
set_pruebas['SVD 1'].describe()

count    20000.000000
mean        96.581427
std         24.831401
min         52.920358
25%         76.701836
50%         90.073607
75%        112.379512
max        157.152840
Name: SVD 1, dtype: float64

In [7]:
columnas = columnas + ['SVD 1', 'SVD 2']

In [8]:
boosting_type = 'goss'
n_estimators = 100000
learning_rate = 2.9

lgbm = lgb.LGBMRegressor(n_estimators = n_estimators, boosting_type = boosting_type, learning_rate = learning_rate)
lgbm.fit(set_entrenamiento_datos, set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: lgbm.predict([x])[0],axis = 1)
precision = lgbm.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 92.14 % , error = 1094773934.55


In [9]:
res = []

In [ ]:
boosting = ['gbdt', 'goss', 'dart']
estimators = [1000, 10000, 100000]
learning_rate = [0.1, 0.3, 0.5]

for b in boosting:
    for e in estimators:
        for lr in learning_rate:
                    
            lgbm = lgb.LGBMRegressor(boosting_type = b, n_estimators = e, learning_rate = lr)
            lgbm.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
            set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply\
                    (lambda x: lgbm.predict([x])[0],axis = 1)
            precision = lgbm.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
            error = mean_squared_error(set_pruebas.price_aprox_usd,set_pruebas.resultado)
            res.append((b, e, lr, precision, error))
            print(b,' - ',e,' - ',lr)

#### Los resultados faltantes con dart se descartaron debido al elevedo tiempo de entrenamiento

In [12]:
for r in res:
    print("Boosting_type = {}, n_estimators = {}, learning_rate = {}, precision = {:.2f} % , error = {}"\
          .format(r[0],r[1],r[2],r[3],r[4]))

Boosting_type = gbdt, n_estimators = 1000, learning_rate = 0.1, precision = 46.00 % , error = 7517061110.72
Boosting_type = gbdt, n_estimators = 1000, learning_rate = 0.3, precision = 45.98 % , error = 7520237244.12
Boosting_type = gbdt, n_estimators = 1000, learning_rate = 0.5, precision = 45.94 % , error = 7525505455.55
Boosting_type = gbdt, n_estimators = 10000, learning_rate = 0.1, precision = 76.46 % , error = 3276631087.32
Boosting_type = gbdt, n_estimators = 10000, learning_rate = 0.3, precision = 75.76 % , error = 3374859215.08
Boosting_type = gbdt, n_estimators = 10000, learning_rate = 0.5, precision = 76.47 % , error = 3274709904.0
Boosting_type = gbdt, n_estimators = 100000, learning_rate = 0.1, precision = 89.23 % , error = 1498608977.3
Boosting_type = gbdt, n_estimators = 100000, learning_rate = 0.3, precision = 89.24 % , error = 1498475513.35
Boosting_type = gbdt, n_estimators = 100000, learning_rate = 0.5, precision = 89.27 % , error = 1493615099.85
Boosting_type = goss,

In [13]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[4] < min_error:
        min_error = r[4]
        tupla_min_error = r
    if r[3] > max_precision:
        max_precision = r[3]
        tupla_max_precision = r
        
print("Mayor precision = Boosting_type = {}, n_estimators = {}, learning_rate = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3],tupla_max_precision[4]))
print("Menor error = Boosting_type = {}, n_estimators = {}, learning_rate = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3],tupla_min_error[4]))

Mayor precision = Boosting_type = goss, n_estimators = 100000, learning_rate = 0.5, precision = 92.14 % , error = 1094565101.87
Menor error = Boosting_type = goss, n_estimators = 100000, learning_rate = 0.5, precision = 92.14 % , error = 1094565101.87


In [18]:
boosting_type = 'goss'
n_estimators = 100000
learning_rate = 2.9

In [19]:
lgbm = lgb.LGBMRegressor(n_estimators = n_estimators, boosting_type = boosting_type, learning_rate = learning_rate)
lgbm.fit(set_entrenamiento_datos, set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: lgbm.predict([x])[0],axis = 1)
precision = lgbm.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 92.13 % , error = 1095184110.6446629


# calculo con los verdaderos datos a analizar

In [20]:
analizar = pd.read_csv("../properati_dataset_modificado.csv")

In [21]:
columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
analisis = svd.transform(analizar.loc[:,columnas])
analisis = pd.DataFrame(analisis)

In [22]:
analizar.loc[:,'SVD 1'] = analisis.loc[:,0]
analizar.loc[:,'SVD 2'] = analisis.loc[:,1]
analizar['SVD 1'].describe()

count    1.416600e+04
mean     3.301517e+02
std      2.023766e+04
min      4.621164e+01
25%      8.041788e+01
50%      1.004089e+02
75%      1.568249e+02
max      2.405964e+06
Name: SVD 1, dtype: float64

In [23]:
columnas += ['SVD 1', 'SVD 2']
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: lgbm.predict([x])[0],axis = 1)
analizar.price_usd.describe()

count     14166.000000
mean     174603.495621
std      121008.662890
min      -82950.371115
25%       98424.253866
50%      141683.573214
75%      208513.166014
max      928491.881348
Name: price_usd, dtype: float64

In [24]:
analizar.loc[:, 'price_usd'] = analizar.loc[:, 'price_usd'].apply(lambda x: abs(x))
analizar.price_usd.describe()

count     14166.000000
mean     174702.098381
std      120866.254873
min         523.545630
25%       98424.253866
50%      141683.573214
75%      208513.166014
max      928491.881348
Name: price_usd, dtype: float64

In [25]:
resultado = analizar.loc[:,['id','price_usd']]

In [26]:
resultado.to_csv('resultados/Lightgbm_menos_datos_svd_agregada.csv', index = False)

In [27]:
joblib.dump(lgbm, 'algoritmos/Lightgbm_svd_agregada.pkl')

['algoritmos/Lightgbm_svd_agregada.pkl']

# Usando muchos estimadores definitivo

In [2]:
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

In [3]:
propiedades = pd.read_csv('../../datos_con_fecha.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', \
            'pileta', 'cochera','transporte','comercio', 'servicios', 'doble piso', 'jardin','fecha']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [4]:
inicio = time.strftime("%X")

lgbm = lgb.LGBMRegressor(boosting_type = 'goss', learning_rate = 0.5, n_estimators = 200000) 
lgbm.fit(set_entrenamiento_datos,set_entrenamiento_resultado)

fin = time.strftime("%X")

print("Tiempo: {} --- {}".format(inicio,fin))

Tiempo: 18:37:43 --- 19:20:00


In [5]:
analizar = pd.read_csv("../properati_dataset_modificado2.csv")

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: abs(lgbm.predict([x])[0]),axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Lightgbm_200kEst_definitivo_confecha.csv', index = False)
joblib.dump(lgbm, 'algoritmos/Lightgbm_200kEst_definitivo_confecha.pkl')

['algoritmos/Lightgbm_200kEst_definitivo_confecha.pkl']

# Usando muchos estimadores definitivo sin fecha

In [6]:
sup_min = 10
sup_max = 300
lat_min = -35
lat_max = -34
lon_min = -59
lon_max = -58
precio_min = 30000
precio_max = 3000000

In [7]:
propiedades = pd.read_csv('../../datos_con_fecha.csv')

propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()) & (propiedades.Year >=2017)\
                        & (propiedades.price_aprox_usd <= precio_max) & (propiedades.price_aprox_usd >= precio_min) &\
                         (propiedades.superficie <= sup_max) & (propiedades.superficie >= sup_min) &\
                      (propiedades.lat <= lat_max) & (propiedades.lat >= lat_min) &\
                       (propiedades.lon <= lon_max) & (propiedades.lon >= lon_min),:]

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', \
            'pileta', 'cochera','transporte','comercio', 'servicios', 'doble piso', 'jardin']

set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [8]:
inicio = time.strftime("%X")

lgbm = lgb.LGBMRegressor(boosting_type = 'goss', learning_rate = 0.5, n_estimators = 200000) 
lgbm.fit(set_entrenamiento_datos,set_entrenamiento_resultado)

fin = time.strftime("%X")

print("Tiempo: {} --- {}".format(inicio,fin))

Tiempo: 19:36:11 --- 20:18:37


In [10]:
analizar = pd.read_csv("../properati_dataset_modificado2.csv")

analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: abs(lgbm.predict([x])[0]),axis = 1)

resultado = analizar.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Lightgbm_200kEst_definitivo_sinfecha.csv', index = False)
joblib.dump(lgbm, 'algoritmos/Lightgbm_200kEst_definitivo_sinfecha.pkl')

['algoritmos/Lightgbm_200kEst_definitivo_sinfecha.pkl']